# Analyse de données de films

Nous proposons de travailler sur des données décrivant des films. Les possibilités sont larges et vous êtes évalués sur vos propositions et votre méthodologie plus que sur vos résultats.

Les données de départ sont disponibles sur:
https://grouplens.org/datasets/movielens/
au format CSV. 

Nous nous intéresserons en particulier au jeu de données: **MovieLens 20M Dataset**. Dans ce jeu de données, vous disposez entre autre de:
* Idendifiant du film dans IMdb et TMdb (ça sera important ensuite)
* Catégorie(s) du film
* Titre du film
* Notes données par les internautes aux films

Afin de rendre le projet plus intéressant, nous ajoutons des données sur les acteurs et producteurs associés aux films (récupéré sur TMdb). Ces données sont disponibles sur les liens suivants:

http://webia.lip6.fr/~guigue/film_v2.pkl <br>
http://webia.lip6.fr/~guigue/act_v2.pkl <br>
http://webia.lip6.fr/~guigue/crew_v2.pkl

Ces fichiers contiennent respectivement : une nouvelle description des films (dont l'identifiant TMdb et la note moyenne donnée par les internautes, la date de sortie,...), une description des acteurs de chaque film et une description des équipes (scénariste, producteur, metteur en scène) pour chaque film.

Ces données sont des listes de taille 26908, chaque élément de la liste correspondant à un dictionnaire dont vous étudierez les clés pour récupérer les informations utiles.

**ATTENTION** Les contraintes de récupération d'informations en ligne font que la base MovieLens compte 27278 films mais les fichiers ci-dessus n'en comptent que 26908. Le plus simple est probablement d'éliminer les films de MovieLens qui ne sont pas dans cette seconde base.

## Consignes générales pour l'analyse des données

Vous devez proposer plusieurs analyses des données, qui devront à minima utiliser les
 techniques suivantes:
 
1. Mettre en forme les données pour identifier les acteurs et les catégories, les indexer
1. Traiter au moins un problème de régression supervisé (par exemple la prédiction de la note moyenne donnée à un film par les internautes).
1. Traiter au moins un problème de classification supervisé (par exemple la prédiction de la catégorie d'un film)
1. Utiliser les données catégorielles (catégories, acteurs,...) de manière discrète ET de manière coninue (*dummy coding*) dans des approches différentes
1. Proposer au moins une approche de catégorisation non supervisée (pour regrouper les acteurs par exemple)
1. Mener une campagne d'expérience permettant de comparer les performances sur un problème en fonction des valeurs d'un paramètre (et donc, in fine, trouver la meilleure valeur du paramètre)
1. Proposer quelques illustrations

# Régression supervisée

## Note moyenne des films:
#### Features:
1) Budget

2) 

# Classification supervisée

## Catégorie d'un film:
#### Features:
1)

# Clustering non supervisé

## Affinité des acteurs:
#### Features:
1) Catégories où ont joué les acteurs associés

2) ...

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle as pkl


## Chargement des données (base MovieLens + enrichissements)

In [2]:
fname = "movielens/act_v2.pkl"
acteurs = pkl.load(open(fname, "rb"))
fname = "movielens/film_v2.pkl" 
movie_bis = pkl.load(open(fname, "rb"))
fname = "movielens/crew_v2.pkl" 
crew = pkl.load(open(fname, "rb"))

In [3]:
movies = pd.DataFrame(data=pd.read_csv("movielens/movies.csv", sep=","))
movies_meta = pd.DataFrame(data=pd.read_csv("movielens/movies_metadata.csv", sep=","))
scores = pd.DataFrame(data=pd.read_csv("movielens/genome-scores.csv", sep=","))
ratings = pd.DataFrame(data=pd.read_csv("movielens/ratings.csv", sep=","))
links = pd.DataFrame(data=pd.read_csv("movielens/links.csv", sep=","))
tags = pd.DataFrame(data=pd.read_csv("movielens/tags.csv", sep=","))
genome_tags =pd.DataFrame(data=pd.read_csv("movielens/genome-tags.csv", sep=","))

/home/angelo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
movies_meta.drop(['homepage', 'poster_path', 'status', 'tagline', 'adult'], inplace=True, axis=1)

In [6]:
import itertools
merged_actors = list(itertools.chain(*acteurs))
merged_crew =  list(itertools.chain(*crew))

In [7]:
df_actors = pd.DataFrame(merged_actors)
df_crew = pd.DataFrame(merged_crew)
df_movies = pd.DataFrame(movie_bis)

In [8]:
df_actors.drop(['profile_path'], inplace=True, axis=1)
df_crew.drop(['profile_path'], inplace=True, axis=1)
df_movies.drop(['backdrop_path'], inplace=True, axis=1)
df_movies.drop(['poster_path'], inplace=True, axis=1)

In [9]:
print(df_movies.isnull().sum())
print(df_crew.isnull().sum())
print(df_actors.isnull().sum())

adult                0
genre_ids            0
id                   0
original_language    0
original_title       0
overview             0
popularity           2
release_date         2
title                0
video                0
vote_average         0
vote_count           0
dtype: int64
credit_id     0
department    0
gender        0
id            0
job           0
name          0
dtype: int64
cast_id      0
character    0
credit_id    0
gender       0
id           0
name         0
order        0
dtype: int64


In [10]:
df_movies.dropna(inplace=True)

In [11]:
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
# the release year for movies w/o one was set to 0
#movies['year'] = pd.to_numeric(movies['year'], errors='raise').fillna(0).astype(np.int64)
movies['title'] = movies.title.str.replace("\s\((\d{4})\)","")

In [12]:
print(movies.isnull().sum())

movieId      0
title        0
genres       0
year       327
dtype: int64


In [13]:
print(tags.isnull().sum())

userId        0
movieId       0
tag          16
timestamp     0
dtype: int64


In [14]:
movies.dropna(inplace=True)
tags.dropna(inplace=True)

In [15]:
ratings["timestamp"] =pd.to_datetime(ratings["timestamp"], unit='s')
ratings["timestamp"] = ratings["timestamp"].dt.date

In [16]:
ratings.tail()

,userId,movieId,rating,timestamp
27753439,283228,8542,4.5,2013-09-22
27753440,283228,8712,4.5,2013-09-22
27753441,283228,34405,4.5,2013-09-22
27753442,283228,44761,4.5,2012-11-29
27753443,283228,54286,4.5,2012-11-29


In [17]:
movies.sort_values(by='movieId', inplace=True)
ratings.sort_values(by='movieId', inplace=True)
tags.sort_values(by='movieId', inplace=True)
links.sort_values(by='movieId', inplace=True)
scores.sort_values(by='movieId', inplace=True)


In [18]:
df = pd.merge(movies,ratings,on='movieId')

In [19]:
df.head()

,movieId,title,genres,year,userId,rating,timestamp
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,220887,4.5,2004-08-10
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,240345,3.5,2007-05-06
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,32915,5.0,1997-01-15
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,37924,4.0,1997-05-23
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,30387,4.0,2016-02-02


In [20]:
ratingCount = pd.DataFrame(df.groupby('title', as_index = False)['rating'].count().rename(columns={'rating' : 'ratingCount'}))

In [21]:
ratingCount.sort_values('ratingCount',ascending=False).head()

,title,ratingCount
36396,"Shawshank Redemption, The",97999
15416,Forrest Gump,97040
33023,Pulp Fiction,92406
36767,"Silence of the Lambs, The",87899
26613,"Matrix, The",84545


In [22]:
ratingMean = pd.DataFrame(df.groupby('title', as_index = False)['rating'].mean().rename(columns={'rating' : 'ratingMean'}))
ratingMean.sort_values('ratingMean',ascending=False).head(5)

,title,ratingMean
5832,Blanche Fury,5.0
39958,Temple of the White Elephant,5.0
46031,Top Secret Rosies: The Female 'Computers' of WWII,5.0
687,8 Murders a Day,5.0
38872,Styx,5.0


In [23]:
ratings_count_mean = pd.merge(ratingCount,ratingMean, on='title')
ratings_count_mean.sort_values('ratingCount',ascending=False).head(5)

,title,ratingCount,ratingMean
36396,"Shawshank Redemption, The",97999,4.424188
15416,Forrest Gump,97040,4.056585
33023,Pulp Fiction,92406,4.173971
36767,"Silence of the Lambs, The",87899,4.151412
26613,"Matrix, The",84545,4.149695


In [24]:
genres_unique = pd.DataFrame(movies.genres.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre'])

In [26]:
genres_unique

,genre
0,Adventure
1,Animation
2,Children
3,Comedy
4,Fantasy
5,Romance
6,Drama
7,Action
8,Crime
9,Thriller


In [27]:
movies_genres = movies.join(movies.genres.str.get_dummies().astype(bool))
movies_genres.drop('genres', inplace=True, axis=1)

In [28]:
movies_genres.head()

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,False,False,True,True,True,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2,Jumanji,1995,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,Grumpier Old Men,1995,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
3,4,Waiting to Exhale,1995,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
4,5,Father of the Bride Part II,1995,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


## Feature engineering
A vous de créer les caractéristiques de description des données qui permettront d'améliorer les performances dans les tâches que vous aurez choisi d'aborder dans le projet.

In [29]:
# Faire un dictionnaire avec tous les acteurs (acteur => indice)
# + un dictionnaire inversé (indice => acteur)
actor_index = dict()
index_actor = dict()
for film in acteurs:
    for act in film:
        # affecte une valeur à une clé si la clé n'est pas utilisé
        res = actor_index.setdefault(act['name'], len(actor_index))
        if res == len(actor_index)-1:
            index_actor[len(actor_index)-1] = act['name']

# Exemple de transformation supplémentaire
# Dans combien de films de base joue Tom Hanks? (Réponse 57)
# Dans combien de comédies...

# => On voit qu'il est possible de créer facilement des nouvelles caractéristiques qui
# apporteront des informations utiles pour certaines tâches

In [32]:
index_actor[0], actor_index["Tom Hanks"]

('Tom Hanks', 0)

In [31]:
genres = dict()
genres_inv = dict()
for g in genres_unique.itertuples():
    # affecte une valeur à une clé si la clé n'est pas utilisée
    res = genres.setdefault(g.genre, len(genres))
    if res == len(genres)-1:
        genres_inv[len(genres)-1] = g.genre

In [32]:
genres["Fantasy"]

4

In [33]:
df_actors.sort_values(by='id', inplace=True)

In [34]:
df_movies.sort_values(by='id', inplace=True)

In [35]:
df_actors.head()

,cast_id,character,credit_id,gender,id,name,order
267934,11,Himself,56757ade92514179d4002e15,2,1,George Lucas,7
182264,78,Baron Papanoida,584a4128c3a368141f01b620,2,1,George Lucas,24
182650,13,Himself,5be613610e0a263bf80039c4,2,1,George Lucas,11
8009,5,Disappointed Man,52fe4235c3a36847f800c2f7,2,1,George Lucas,21
339106,1,Himself,52fe48799251416c9108db99,2,1,George Lucas,0


In [36]:
df_movies.loc[df_movies.original_title == 'Toy Story']

,adult,genre_ids,id,original_language,original_title,overview,popularity,release_date,title,video,vote_average,vote_count
0,False,"[16, 35, 10751]",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",22.773,1995-10-30,Toy Story,False,7.9,9550


In [37]:
movies.loc[movies.title == 'Toy Story']

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995


In [38]:
df_movies.loc[df_movies.title == 'Toy Story']

,adult,genre_ids,id,original_language,original_title,overview,popularity,release_date,title,video,vote_average,vote_count
0,False,"[16, 35, 10751]",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",22.773,1995-10-30,Toy Story,False,7.9,9550


In [39]:
acteurs[0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/xxPMucou2wRDxLrud8i2D4dsywh.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/dDbtWMGdhatUjCIYolc312R2ygu.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [40]:
df_actors.loc[df_actors.name == 'Tim Allen']

,cast_id,character,credit_id,gender,id,name,order
356723,7,Buzz Lightyear,52fe4dbec3a368484e1fac15,2,12898,Tim Allen,1
266660,9,Buzz Lightyear (voice),52fe433f9251416c75009169,2,12898,Tim Allen,1
5988,10,Santa Claus / Scott Calvin,52fe44379251416c7502ce9b,2,12898,Tim Allen,0
152772,1,Luther Krank,52fe458a9251416c7505a0a7,2,12898,Tim Allen,0
225420,2,Chet Frank,52fe44dc9251416c750437b5,2,12898,Tim Allen,1
195611,22,Dave Douglas,52fe431a9251416c75003823,2,12898,Tim Allen,0
278490,1,Tommy,52fe45a69251416c910399e7,2,12898,Tim Allen,0
28496,11,Michael Cromwell,52fe44f8c3a36847f80b4f77,2,12898,Tim Allen,0
389622,1,Buzz Lightyear (voice),52fe48509251416c91087f5d,2,12898,Tim Allen,1
197625,68,Buzz Lightyear Car (voice),550da0af9251414695005f49,2,12898,Tim Allen,23


In [41]:
movies.loc[movies.title == 'Pulp Fiction']

,movieId,title,genres,year
293,296,Pulp Fiction,Comedy|Crime|Drama|Thriller,1994


In [51]:
df_movies.loc[df_movies.original_title == 'Pulp Fiction']

,adult,genre_ids,id,original_language,original_title,overview,popularity,release_date,title,video,vote_average,vote_count
293,False,"[53, 80]",680,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",29.059,1994-09-10,Pulp Fiction,False,8.4,14296


In [49]:
genres_vals = df_movies.genre_ids.values
genres_vals = [el for sub in genres_vals for el in sub]
un_genre_vals = np.unique(genres_vals)

In [50]:
un_genre_vals

array([   12,    14,    16,    18,    27,    28,    35,    36,    37,
          53,    80,    99,   878,  9648, 10402, 10749, 10751, 10752,
       10770])

## problème rencontré en essayant de faire des dictionnaires Films: {actors} et Actors:{categories} :
## Je ne comprends pas a quel genre correspond chaque genre_ids et comment trouver les films dans lesquels chaque acteur a jouer